# СРС №1. Работа с базой данных в программном коде
## База данных котов и их хозяев.
Коты любят кучковаться, поэтому объединяются в прайды, а их хозяева отслеживают эти прайды и дают им названия. У каждого кота может быть несколько хозяев.
Обо всех хозяевах известна информация в какой семье они состоят, при этом один человек может состоять в нескольких семьях.
Описание таблиц:
cat - таблица котов (первичный ключ, неуникальная кличка, порода, прайд, возраст)
breed - таблица пород  (первичный ключ, уникальное название)
pride - таблица прайдов (первичный ключ, уникальное название)
owner - таблица хозяев (первичный ключ, уникальное ФИО)
family - таблица семей (первичный ключ, уникальное название)
cat_owner - таблица пересечений котов и хозяев (ссылка на хозяина, ссылка на кота)
owner_family - таблица пересечений хозяев и семей (ссылка на хозяина, ссылка на семью)

## Задание.
1. Реализовать описанную выше структуру базы данных в СУБД MySQL
2. На любом языке программирования реализовать программный код, отвечающий за генерацию тестовых данных.
Тестовые данные должны отвечать следующим условиям:
	Не менее 30 котов
	Не менее 30 пород
	Не менее 15 прайдов
	Не менее 40 хозяев
	Не менее 15 семей
	Клички должны быть неуникальные
	Названия пород, прайдов, семей должны быть уникальными
	Должны присутствовать как минимум 3 кота с 2 хозяевами, как минимум 5 котов с 3 хозяевами
	Как минимум 2 породы и 3 прайда должны быть без котов, как минимум 2 семьи без хозяев, как минимум 2 кота должны иметь несколько хозяев из одной и той же семьи.
3. Реализовать программный код с текстовым или графическим интерфейсом, позволяющий получить ответ на перечисленные ниже вопросы:
	1. Коты каких пород встречаются в данной базе?
	2. В каких прайдах нет ни одного кота?
	3. В каких семьях нет ни одного кота?
	4. Какой средний возраст котов породы "сфинкс"?
	5. В каких семьях наибольшее количество котов прайда "Бесславные ублюдки"?
	6. Сколько человек в семье "Рабинович"?
	7. Сколько котов в семье "Энгельс"?
	8. Какой прайд самый многочисленный?
	9. Хозяином котов каких пород является Карл Маркс?
	10. Какая кличка самая популярная?

In [1]:
import mysql.connector
import pandas as pd
import random 
from faker import Faker
import pandas as pd
from mysql.connector import connect, Error
#import getpass
fake = Faker('ru_RU')
r_host = "localhost"
r_user = "root"
r_password = "aryuna"
r_db = "a_try"
connection = connect(
    host=r_host, 
    user=r_user,
    password=r_password,
    database = r_db,
)
#db_cur = connection.cursor(buffered = True)
db_cur = connection.cursor()

In [2]:
Cat = """
CREATE TABLE IF NOT EXISTS cat(
    cat_id INT PRIMARY KEY NOT NULL,
    cat_name VARCHAR(100) NOT NULL,
    cat_breed INT NOT NULL,
    cat_pride INT NOT NULL,
    cat_age INT NOT NULL
)
"""
Breed = """
CREATE TABLE IF NOT EXISTS breed(
    breed_id INT PRIMARY KEY NOT NULL,
    breed_name VARCHAR(100) NOT NULL UNIQUE
)
"""
Pride = """
CREATE TABLE IF NOT EXISTS pride(
    pride_id INT PRIMARY KEY NOT NULL,
    pride_name VARCHAR(100) NOT NULL UNIQUE
)
"""
Owners = """
CREATE TABLE IF NOT EXISTS owner(
    owner_id INT PRIMARY KEY NOT NULL,
    owner_name VARCHAR(200) NOT NULL UNIQUE
)
"""
Families = """
CREATE TABLE IF NOT EXISTS family(
    family_id INT PRIMARY KEY NOT NULL,
    family_name VARCHAR(100) NOT NULL UNIQUE
)
"""
create_cat_owner  = """
CREATE TABLE IF NOT EXISTS cat_owner(
    cat_id INT NOT NULL,
    owner_id INT NOT NULL,
    PRIMARY KEY (cat_id, owner_id)
)
"""
create_owner_family  = """
CREATE TABLE IF NOT EXISTS owner_family(
    owner_id INT NOT NULL,
    family_id INT NOT NULL,
    PRIMARY KEY (owner_id, family_id)
)
"""

show_t = "SHOW TABLES FROM a_try"
db_cur.execute(show_t)
print(db_cur.fetchall()) # в курсоре сейчас элементы

[('breed',), ('cat',), ('cat_owner',), ('family',), ('owner',), ('owner_family',), ('pride',)]


In [4]:
d = pd.read_csv('D:/Programmes/my stupid or not attempts to make a code/DB/cats.csv', sep=',', header=None, low_memory=False) #достаём породы из какого-то кагловского датасета
headers = d.iloc[0]
breeds = pd.DataFrame(d.values[1:], columns=headers)
breeds = breeds['breed'].unique()
breed_num = random.randint(30, len(breeds))
cat_num = random.randint(30, 100)
pride_num = random.randint(15, 100)
owner_num = random.randint(40, 100)
family_num = random.randint(15, 100)
breeds_for_table = random.sample(sorted(breeds), breed_num) #само генерирование пород (смешивание)
print(breed_num)
print(cat_num)
print(pride_num)
print(owner_num)
print(family_num)

35
61
26
70
36


In [5]:
drop = 'DROP TABLE IF EXISTS cat_owner, owner_family, breed, pride, owner, family, cat'
db_cur.execute(drop); connection.commit()

In [6]:
db_cur.execute(Cat)
db_cur.execute(Breed)
db_cur.execute(Pride)
db_cur.execute(Owners)
db_cur.execute(Families)
db_cur.execute(create_cat_owner)
db_cur.execute(create_owner_family)
connection.commit()

In [7]:
def FILL_BREED(cur, n, breeds_for_table):
    for i in range(n):
        cur.execute(f"INSERT IGNORE INTO breed (breed_id, breed_name) VALUES ({i}, '{breeds_for_table[i]}')")
    connection.commit()
    cur.execute("SELECT breed_name from breed WHERE breed_name = 'Sphynx - Hairless Cat'")
    p = cur.fetchone()
    if not (p):
        cur.execute(f"INSERT INTO breed (breed_id, breed_name) VALUES ({n}, 'Sphynx - Hairless Cat')")
        n += 1
        connection.commit()
def FILL_PRIDE(cur, n, fake):
    for i in range(n):
        cur.execute(f"INSERT IGNORE INTO pride (pride_id, pride_name) VALUES ({i}, '{fake.unique.city_name()}')")
    #в честь городов прайды названы
    connection.commit(); fake.unique.clear()
    cur.execute("SELECT pride_name from pride WHERE pride_name = 'Бесславные ублюдки'")
    p = cur.fetchone()
    if not (p):
        _besub = random.randint(0, n)
        cur.execute(f"UPDATE pride SET pride_name = 'Бесславные ублюдки' WHERE pride_id = {_besub} OR pride_name = 'Бесславные ублюдки'")
        connection.commit()
def FILL_OWNER(cur, n, fake):
    for i in range(n):
        cur.execute(f"INSERT IGNORE INTO owner (owner_id, owner_name) VALUES ({i}, '{fake.name()}')")
    connection.commit()
    cur.execute("SELECT owner_name from owner WHERE owner_name = 'Карл Маркс'")
    p = cur.fetchone()
    if not (p):
        _marx = random.randint(0, n)
        cur.execute(f"UPDATE owner SET owner_name = 'Карл Маркс' WHERE owner_id = {_marx} OR owner_name = 'Карл Маркс'")
        connection.commit()
def FILL_FAMILY(cur, n, fake):
    for i in range(n):
        cur.execute(f"INSERT IGNORE INTO family (family_id, family_name) VALUES ({i}, '{fake.unique.last_name()}')")
    connection.commit(); fake.unique.clear()
    cur.execute("SELECT family_name from family WHERE family_name = 'Рабинович'")
    r = cur.fetchone()
    cur.execute("SELECT family_name from family WHERE family_name = 'Энгельс'")
    e = cur.fetchone()
    _rabi = random.randint(0, n)
    _engels = random.choice([a for a in range(n) if a != _rabi]) #чтоб точно не совпали
    if not (r):
        cur.execute(f"UPDATE family SET family_name = 'Рабинович' WHERE family_id = {_rabi} OR family_name = 'Рабинович'")
        connection.commit()
    if not (e):
        cur.execute(f"UPDATE family SET family_name = 'Энгельс' WHERE family_id = {_engels} OR family_name = 'Энгельс'")
        connection.commit()
def FILL_CAT(cur, n, fake):
    embreed = random.sample(range(n), random.randint(2, 5))
    empride = random.sample(range(n), random.randint(3, 5))
    cur.execute('SELECT breed_id FROM breed')
    breed_ids = [a[0] for a in cur.fetchall() if a not in embreed]
    cur.execute('SELECT pride_id FROM pride')
    pride_ids = [a[0] for a in cur.fetchall() if a not in empride]
    for i in range(n):
        cur.execute(f"""INSERT IGNORE INTO cat (cat_id, cat_name, 
    cat_breed, cat_pride, cat_age) VALUES ({i}, '{fake.first_name()}', 
    {random.choice(breed_ids)}, {random.choice(pride_ids)}, {random.randint(1, 37)})
    """)

In [8]:
def FILL_CAT_OWNER(cur):
    cur.execute('SELECT owner_id FROM owner')
    owner_ids = [a[0] for a in cur.fetchall()]
    cur.execute('SELECT cat_id FROM cat')
    cat_ids = [a[0] for a in cur.fetchall()]
    for i in cat_ids:
        cur.execute(f"INSERT IGNORE INTO cat_owner (cat_id, owner_id) VALUES ({i}, {random.choice(owner_ids)})")
        connection.commit()
    cur.execute('''SELECT COUNT(cat_id) from cat WHERE cat_id IN 
    (SELECT cat_id from cat_owner GROUP BY cat_id HAVING COUNT(owner_id)=2)''')
    two = cur.fetchone()[0]
    cur.execute('''SELECT COUNT(cat_id) from cat WHERE cat_id IN 
    (SELECT cat_id from cat_owner GROUP BY cat_id HAVING COUNT(owner_id)=3)''')
    three = cur.fetchone()[0]
    if (two < 2):
        two = random.randint(3, 13)
        cats_with_two = random.sample(cat_ids, two)
        for i in cats_with_two:
            cur.execute(f"INSERT IGNORE INTO cat_owner (cat_id, owner_id) VALUES ({i}, {random.choice(owner_ids)})")
            connection.commit()
    if (three < 5):
        three = random.randint(5, 9)
        cats_with_three = random.sample(cat_ids, three)
        for i in cats_with_three:
            cur.execute(f'''SELECT count(owner_id) FROM cat_owner GROUP BY cat_id 
        HAVING count(owner_id) < 3 AND cat_id = {i}''') #коты с < 3 хозяевами
            o = cur.fetchone()[0]
            while (o < 3):
                cur.execute(f"INSERT IGNORE INTO cat_owner (cat_id, owner_id) VALUES ({i}, {random.choice(owner_ids)})")
                connection.commit()
                o += 1

In [9]:
def FILL_OWNER_FAMILY(cur):
    cur.execute('SELECT owner_id FROM owner')
    tmp = cur.fetchall()
    no_fam = random.randint(2, 5)
    #ownofam = random.sample(tmp, no_fam)
    owner_ids = [a[0] for a in tmp] #if a not in ownofam]
    cur.execute('SELECT family_id FROM family')
    family_ids = [a[0] for a in cur.fetchall()]
    for i in owner_ids:
        cur.execute(f"INSERT IGNORE INTO owner_family (owner_id, family_id) VALUES ({i}, {random.choice(family_ids)})")
        connection.commit()
#    cur.execute('''SELECT COUNT(*) FROM family WHERE family_id NOT IN 
#    (SELECT family_id from owner_family)''')
#    no_fam = *cur.fetchone()[0]
#    if (no_fam < 2):
#        no_fam = random.randint(2, 10)
#        ownofam = random.sample(owner_ids, no_fam)
#            for i in ownofam:
#                cur.execute(f"DELETE FROM owner_family WHERE owner_id = {i}")
#                connection.commit()
    cur.execute('''SELECT COUNT(*) FROM (SELECT owf.family_id, co.cat_id, COUNT(*) as cnt 
    FROM owner_family owf JOIN cat_owner co ON co.owner_id = owf.owner_id 
    GROUP BY family_id, co.cat_id HAVING cnt>1) same''')
    sameowf = int(cur.fetchone()[0])
    if (sameowf < 2):
        sameowf = random.randint(2, 6)
        cur.execute('SELECT cat_id FROM cat_owner GROUP BY cat_id HAVING count(owner_id) > 1')
        cat_same_owner = [a[0] for a in cur.fetchall()] #список котов с > 1 хозяином
        same_fam_cat = random.sample(cat_same_owner, sameowf) #пока просто выбрали несколько котов, у которых будет одна семья
        for i in same_fam_cat:
            cur.execute(f'SELECT owner_id from cat_owner WHERE cat_id = {i}')
            owners_from_same_family = [a[0] for a in cur.fetchall()] #список самих хозяев 
            samefamily = random.choice(family_ids)
            for a in owners_from_same_family:
                cur.execute(f"UPDATE owner_family SET family_id = {samefamily} WHERE owner_id = {a}") 
                connection.commit()

In [10]:
FILL_BREED(db_cur, breed_num, breeds_for_table)
FILL_PRIDE(db_cur, pride_num, fake)
FILL_OWNER(db_cur, owner_num, fake)
FILL_FAMILY(db_cur, family_num, fake)
FILL_CAT(db_cur, cat_num, fake)
FILL_CAT_OWNER(db_cur)
FILL_OWNER_FAMILY(db_cur)

In [11]:
def SHOW(table, cur):
    cur.execute(f'SELECT * FROM {table}')
    print(cur.fetchall())
    print()
def COUNT(table, cur):
    cur.execute(f'SELECT COUNT(*) FROM {table}')
    print(cur.fetchall())
    print()
def SHOW10(table, cur):
    cur.execute(f'SELECT * FROM {table} LIMIT 10')
    print(cur.fetchall())
    print()
def SHOW_BREEDS(cur):
    cur.execute("SELECT DiSTINCT breed.breed_name FROM cat INNER JOIN breed ON cat.cat_breed=breed.breed_id")
    print(cur.fetchall())
    print()
def SHOW_NO_CATS_PRIDES(cur):
    cur.execute('''SELECT pride_name FROM pride WHERE pride_id NOT IN 
    (SELECT cat_pride from cat)''')
    print(cur.fetchall())
    print()
def SHOW_NO_CATS_FAMILIES(cur):
    cur.execute('''SELECT family_name FROM family WHERE family_id NOT IN
    (SELECT owf.family_id FROM owner_family AS owf 
    JOIN cat_owner co ON owf.owner_id=co.owner_id)''')
    print(cur.fetchall())
    print()
def SPHYNX_AVG(cur):
    cur.execute("""SELECT AVG(cat_age) FROM cat JOIN breed b ON 
                b.breed_id = cat.cat_breed 
                AND b.breed_name = 'Sphynx - Hairless Cat'
                """)
    sph = cur.fetchone()[0]
    if not sph:
        print("Сфинксов нет.")
    else:
        print(sph)
    print()
def BESUB(cur):
    cur.execute('''
                WITH t AS(
SELECT f.family_name, COUNT(c.cat_id) AS num FROM family as f JOIN owner_family as owf ON
f.family_id = owf.family_id join cat_owner co ON
co.owner_id=owf.owner_id join cat c On
c.cat_id=co.cat_id join pride p on 
p.pride_id=c.cat_pride AND p.pride_name = 'Бесславные ублюдки'
group by f.family_name
)
SELECT family_name FROM t WHERE num=(SELECT MAX(num) FROM t)
                ''')
    bu = cur.fetchall()
    if not bu:
        print("Нет семей с бесславными ублюдками.")
    else:
        print(bu)
    print()

def CountEngels(cur):
    cur.execute('''
                SELECT count(c.cat_id) FROM cat c
                JOIN cat_owner co on co.cat_id = c.cat_id
                JOIN owner_family owf on co.owner_id = owf.owner_id 
                JOIN family f on owf.family_id = f.family_id 
                AND f.family_name = "Энгельс"
''')
    eng = cur.fetchone()[0]
    if not eng:
        print("У Энгельсов нет котов...")
    else:
        print(eng)
    print()
def CountRabinovich(cur):
    cur.execute('''
        SELECT count(owf.family_id) FROM owner_family as owf 
        JOIN family f ON owf.family_id = f.family_id 
        AND f.family_name = "Рабинович"
        ''')
    rab = cur.fetchone()[0]
    if not rab:
        print("Рабиновичи без людей...")
    else:
        print(rab)
    print()
def BiggestPride(cur):
    cur.execute('''SELECT cat_pride, p.pride_name, COUNT(cat_id) FROM cat 
    JOIN pride p ON pride_id = cat_pride
GROUP BY cat_pride
ORDER BY COUNT(cat_pride) DESC LIMIT 1
''')
    print(cur.fetchall())
    print()
def BreedsMarx(cur):
    cur.execute('''SELECT distinct b.breed_name FROM breed b 
join cat c on b.breed_id=c.cat_breed join cat_owner co on
co.cat_id=c.cat_id join owner o on
o.owner_id=co.owner_id AND o.owner_name='Карл Маркс'
                ''')
    mmm = cur.fetchall()
    if not mmm or len(mmm) == 0:
        cur.execute('''select COUNT(*) FROM cat_owner co JOIN owner o ON o.owner_id = co.owner_id WHERE o.owner_name = "Карл Маркс"''')
        temp = cur.fetchone()[0]
        print(f"У Маркса котов {temp}")
    else:
        for i in mmm:
            print(i)
    print()

def MostPopular(cur):
    cur.execute('''SELECT cat_name, Count(cat_id) 
                FROM cat GROUP BY cat_name 
    HAVING Count(cat_id)  = (SELECT MAX(a.cou) 
    FROM (SELECT count(cat_id) as cou from CAT GROUP BY cat_name) as a);
    ''')
    print(cur.fetchall())
    print()

In [12]:
def Intro(cur):
    print("""Вы пока во вступлении. Введите через пробел номер команды и название таблицы, 
      о которой хотите узнать (cat, breed, pride, owner, family, cat_owner, owner_family)) """)
    print("Когда закончите, введите 'К основным' ")
    tables = ['cat', 'breed', 'pride', 'owner', 'family', 'cat_owner', 'owner_family']
    tab = ''
    while tab != 'К основным':
        tab = input("Введите название интересующей таблицы: ")
        if (any(x == tab for x in tables)):
            #SHOW(tab, db_cur)
            COUNT(tab, db_cur)
            SHOW10(tab, db_cur)
    Main(cur)
def Main(cur):
    tab = ''
    while tab != 'Всё':
        tab = input("Введите число от 1 до 10: ")
        if tab == '1':
            SHOW_BREEDS(db_cur)
        elif tab == '2':
            SHOW_NO_CATS_PRIDES(db_cur)
        elif tab == '3':
            SHOW_NO_CATS_FAMILIES(db_cur)
        elif tab == '4':
            SPHYNX_AVG(db_cur)
        elif tab == '5':
            BESUB(db_cur)
        elif tab == '6':
            CountEngels(db_cur)
        elif tab == '7':
            CountRabinovich(db_cur)
        elif tab == '8':
            BiggestPride(db_cur)
        elif tab == '9':
            BreedsMarx(db_cur)
        elif tab == '10':
            MostPopular(db_cur)
        elif tab == 'Всё':
            print("До свидания")
        elif tab == 'Проверка':
            CHECK_DEMANDS(db_cur)
        else:
            print("Вы ввели что-то не то... ")
            #pass

In [13]:
def CHECK_DEMANDS(cur):
    #cur.execute("""SELECT c.cat_name, co.owner_name from cat c LEFT JOIN cat_owner co 
    #ON c.cat_id = co.cat_id GROUP BY c.cat_name""")
    #"SELECT COUNT(cat_id) from cat WHERE cat_id IN (SELECT cat_id from cat_owner GROUP BY cat_id HAVING COUNT(owner_id)=2)"
    #print(cur.fetchall())
    cur.execute("""SELECT COUNT(*) from breed WHERE breed_id NOT IN (SELECT cat_breed FROM cat)
    """)
    b = int(cur.fetchone()[0])
    print(str(b) + ' пород без котов')
    print()
    cur.execute('''SELECT COUNT(*) FROM pride WHERE pride_id NOT IN 
    (SELECT cat_pride from cat)''')
    p = int(cur.fetchone()[0])
    print(str(p) + " прайдов без котов")
    print()
    cur.execute('''SELECT COUNT(*) FROM family WHERE family_id NOT IN 
    (SELECT family_id from owner_family)''')
    f = int(cur.fetchone()[0])
    print(str(f) + " семей без хозяев")
    print()
    cur.execute('''SELECT COUNT(cat_id) from cat WHERE cat_id IN 
    (SELECT cat_id from cat_owner GROUP BY cat_id HAVING COUNT(owner_id)=2)''')
    two = int(cur.fetchone()[0])
    print(str(two) + " котов с 2-мя хозяевами")
    print()
    cur.execute('''SELECT COUNT(cat_id) from cat WHERE cat_id IN 
    (SELECT cat_id from cat_owner GROUP BY cat_id HAVING COUNT(owner_id)=3)''')
    three = int(cur.fetchone()[0])
    print(str(three) + " котов с 3-мя хозяевами")
    print()
    cur.execute('''SELECT COUNT(*) FROM (SELECT family_id, co.cat_id, COUNT(*) as cnt 
    FROM owner_family owf JOIN cat_owner co ON co.owner_id = owf.owner_id 
    GROUP BY family_id, co.cat_id HAVING cnt>1) t1''')
    sameown = int(cur.fetchone()[0])
    print(str(sameown) + " котов с хозяевами из одной и той же семьи")
    print()

### Вступление:
    1. Вывести количество строк в ...
    2. Вывести первые 10 строк в ...

### Основные вопросы:
	1. Коты каких пород встречаются в данной базе?
	2. В каких прайдах нет ни одного кота?
	3. В каких семьях нет ни одного кота?
	4. Какой средний возраст котов породы "сфинкс"?
	5. В каких семьях наибольшее количество котов прайда "Бесславные ублюдки"?
	6. Сколько человек в семье "Рабинович"?
	7. Сколько котов в семье "Энгельс"?
	8. Какой прайд самый многочисленный?
	9. Хозяином котов каких пород является Карл Маркс?
	10. Какая кличка самая популярная?

In [ ]:
Intro(db_cur)

Вы пока во вступлении. Введите через пробел номер команды и название таблицы, 
      о которой хотите узнать (cat, breed, pride, owner, family, cat_owner, owner_family)) 
Когда закончите, введите 'К основным' 
Введите название интересующей таблицы: К основным
Введите число от 1 до 10: 1
[('York Chocolate',), ('Ragamuffin',), ('Oriental Tabby',), ('Tuxedo',), ('Ocicat',), ('Domestic Long Hair',), ('Singapura',), ('Manx',), ('Munchkin',), ('Maine Coon',), ('Exotic Shorthair',), ('American Shorthair',), ('Ragdoll',), ('Javanese',), ('Snowshoe',), ('Domestic Short Hair',), ('Dilute Tortoiseshell',), ('Siberian',), ('American Bobtail',), ('Cornish Rex',), ('LaPerm',), ('British Shorthair',), ('Russian Blue',), ('Extra-Toes Cat - Hemingway Polydactyl',), ('Havana',), ('Canadian Hairless',), ('Burmese',), ('Oriental Short Hair',), ('Balinese',), ('Calico',), ('Persian',), ('Torbie',), ('Tabby',), ('Somali',), ('Nebelung',), ('Norwegian Forest Cat',), ('Domestic Medium Hair',), ('Turkish Van',

In [14]:
Intro(db_cur)

Вы пока во вступлении. Введите через пробел номер команды и название таблицы, 
      о которой хотите узнать (cat, breed, pride, owner, family, cat_owner, owner_family)) 
Когда закончите, введите 'К основным' 
Введите название интересующей таблицы: cat
[(61,)]

[(0, 'Гурий', 10, 8, 28), (1, 'Лора', 34, 0, 14), (2, 'Майя', 29, 4, 4), (3, 'Аникита', 9, 18, 6), (4, 'Поликарп', 11, 0, 34), (5, 'Емельян', 4, 6, 37), (6, 'Всеволод', 14, 24, 24), (7, 'Лука', 5, 11, 31), (8, 'Мирослав', 15, 16, 33), (9, 'Евфросиния', 22, 2, 27)]

Введите название интересующей таблицы: breed
[(35,)]

[(15, 'Abyssinian'), (11, 'American Wirehair'), (3, 'Bengal'), (14, 'Burmese'), (33, 'Burmilla'), (12, 'Calico'), (1, 'Chinchilla'), (24, 'Dilute Tortoiseshell'), (10, 'Domestic Long Hair'), (0, 'Domestic Medium Hair')]

Введите название интересующей таблицы: pride
[(26,)]

[(22, 'Адлер'), (12, 'Анапа'), (3, 'Арсеньев'), (10, 'Белоярский'), (19, 'Бесславные ублюдки'), (1, 'Гудермес'), (25, 'Екатеринбург'), (17, 'Ка

In [ ]:
db_cur.execute("SELECT COUNT(*) from (SELECT DISTINCT family_name from family) s")
print(db_cur.fetchall())
db_cur.execute("SELECT COUNT(*) from family")
print(db_cur.fetchall())

In [ ]:
db_cur.execute(''' 
SELECT breed_name FROM breed WHERE breed_id NOT IN (SELECT cat_breed from cat)
''')
print(db_cur.fetchall())
#27 10-12:30 1112 аудитория
#составные перв ключи в двойных

In [8]:
db_cur.execute(''' 
SELECT COUNT(breed_name) - COUNT(DISTINCT breed_name) from breed
''')
print(db_cur.fetchall())
db_cur.execute(''' 
SELECT COUNT(pride_name) - COUNT(DISTINCT pride_name) from pride
''')
print(db_cur.fetchall())
db_cur.execute(''' 
SELECT COUNT(family_name) - COUNT(DISTINCT family_name) from family
''')
print(db_cur.fetchall())

[(0,)]
[(0,)]
[(0,)]


In [9]:
db_cur.execute(''' 
SELECT COUNT(cat_name) - COUNT(DISTINCT cat_name) from cat
''')
print(db_cur.fetchall())

[(8,)]
